In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/mnt/software/'
import sys
import gc
# assuming data, models, engine in flicc directory:
flicc_path = os.path.join(os.path.dirname(os.getcwd()), '')
sys.path.append(flicc_path)
import torch
from data import ClimateDataset
from models import ClassificationModel
from engine import Engine

# import warnings
# warnings.filterwarnings("ignore")

In [2]:
best_config = {'bert-base-uncased':{'lr':5.0e-5, 'wd':0.0},
                'roberta-large':{'lr':5.0e-5, 'focalloss':True, 'gamma':8, 'wd':0.0},
                'gpt2':{'lr':5.0e-5, 'wd':0.01},
                'bigscience/bloom-560m':{'lr':5.0e-5, 'focalloss':True, 'gamma':8, 'wd':0.0},
                'facebook/opt-350m':{'lr':1.0e-5, 'wd':0.0},
                'EleutherAI/gpt-neo-1.3B':{'lr':5.0e-5, 'wd':0.0}, 
                'microsoft/deberta-base':{'lr':1.0e-5, 'wd':0.01},
                'microsoft/deberta-v2-xlarge':{'lr':1.0e-5, 'focalloss':True, 'gamma':4, 'wd':0.01}}

In [3]:
results = {'test_acc':[],
           'test_f1':[],
           'eval_acc':[],
           'eval_f1':[],
           'rank, alpha':[],
           'model':[]}

In [4]:
alphas = [8, 16]
ranks = [8, 16]

for model_checkpoint in best_config.keys():
    for a,r in zip(alphas, ranks):
        print(f'Grid search {model_checkpoint}, learning rate {best_config[model_checkpoint]["lr"]}')
        data = ClimateDataset(model_to_train=4,model_checkpoint=model_checkpoint,dataset_url=flicc_path,batch_size=32)
        data.setup_dataloaders()
        model = ClassificationModel(model_checkpoint=data.model_checkpoint,
                                    num_labels=data.num_labels,
                                    lora=True,
                                    alpha=a,
                                    r=r,
                                    dropout=0.0)
        trainer = Engine(epochs=30,labels=data.labels)
        trainer.model = model.model
        trainer.dataset_encoded = data.dataset_encoded
        test_acc, test_f1, eval_acc, eval_f1 = trainer.run(**best_config[model_checkpoint],
                                                            train_dataloader=data.train_dataloader,
                                                            eval_dataloader=data.eval_dataloader,
                                                            test_dataloader=data.test_dataloader,
                                                            early_stop=3)
        results['test_acc'].append(test_acc)
        results['test_f1'].append(test_f1)
        results['eval_acc'].append(eval_acc)
        results['eval_f1'].append(eval_f1)
        results['rank, alpha'].append(r)
        results['model'].append(model_checkpoint)
        print('### '*10)
        print(results)
        print('### '*10)
        with torch.no_grad():
            torch.cuda.empty_cache()
        del data, model, trainer
        gc.collect()

Grid search bert-base-uncased, learning rate 5e-05


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


1 / 30: Train Loss:	2.4386	Val Loss:	2.3611	Accuracy:	0.1554	F1:	0.0359 *
2 / 30: Train Loss:	2.3651	Val Loss:	2.3339	Accuracy:	0.1619	F1:	0.0429 *
3 / 30: Train Loss:	2.3422	Val Loss:	2.2988	Accuracy:	0.1860	F1:	0.0723 *
4 / 30: Train Loss:	2.2950	Val Loss:	2.2318	Accuracy:	0.2276	F1:	0.1018 *
5 / 30: Train Loss:	2.2238	Val Loss:	2.1524	Accuracy:	0.2604	F1:	0.1201 *
6 / 30: Train Loss:	2.1352	Val Loss:	2.0615	Accuracy:	0.2845	F1:	0.1507 *
7 / 30: Train Loss:	2.0462	Val Loss:	1.9746	Accuracy:	0.3129	F1:	0.1741 *
8 / 30: Train Loss:	1.9507	Val Loss:	1.8959	Accuracy:	0.3370	F1:	0.1940 *
9 / 30: Train Loss:	1.8689	Val Loss:	1.8234	Accuracy:	0.3545	F1:	0.2184 *
10 / 30: Train Loss:	1.8023	Val Loss:	1.7666	Accuracy:	0.3851	F1:	0.2433 *
11 / 30: Train Loss:	1.7494	Val Loss:	1.7254	Accuracy:	0.4004	F1:	0.2604 *
12 / 30: Train Loss:	1.6952	Val Loss:	1.6919	Accuracy:	0.4267	F1:	0.2891 *
13 / 30: Train Loss:	1.6663	Val Loss:	1.6681	Accuracy:	0.4354	F1:	0.2930 *
14 / 30: Train Loss:	1.6428	Val Lo

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


1 / 30: Train Loss:	2.4376	Val Loss:	2.3581	Accuracy:	0.1554	F1:	0.0357 *
2 / 30: Train Loss:	2.3594	Val Loss:	2.3196	Accuracy:	0.1707	F1:	0.0524 *
3 / 30: Train Loss:	2.3158	Val Loss:	2.2533	Accuracy:	0.2254	F1:	0.0935 *
4 / 30: Train Loss:	2.2332	Val Loss:	2.1610	Accuracy:	0.2560	F1:	0.1147 *
5 / 30: Train Loss:	2.1414	Val Loss:	2.0686	Accuracy:	0.2779	F1:	0.1435 *
6 / 30: Train Loss:	2.0354	Val Loss:	1.9806	Accuracy:	0.3173	F1:	0.1778 *
7 / 30: Train Loss:	1.9464	Val Loss:	1.8908	Accuracy:	0.3567	F1:	0.2157 *
8 / 30: Train Loss:	1.8515	Val Loss:	1.8104	Accuracy:	0.3742	F1:	0.2428 *
9 / 30: Train Loss:	1.7724	Val Loss:	1.7469	Accuracy:	0.3982	F1:	0.2645 *
10 / 30: Train Loss:	1.7145	Val Loss:	1.7007	Accuracy:	0.4114	F1:	0.2837 *
11 / 30: Train Loss:	1.6641	Val Loss:	1.6616	Accuracy:	0.4289	F1:	0.3010 *
12 / 30: Train Loss:	1.6072	Val Loss:	1.6463	Accuracy:	0.4267	F1:	0.3069 *
13 / 30: Train Loss:	1.5771	Val Loss:	1.6105	Accuracy:	0.4398	F1:	0.3134 *
14 / 30: Train Loss:	1.5452	Val Lo

...

1 / 30: Train Loss:	2.4831	Val Loss:	2.4770	Accuracy:	0.1247	F1:	0.0185 *
2 / 30: Train Loss:	2.4728	Val Loss:	2.4659	Accuracy:	0.1247	F1:	0.0185
3 / 30: Train Loss:	2.4629	Val Loss:	2.4560	Accuracy:	0.1247	F1:	0.0185
4 / 30: Train Loss:	2.4533	Val Loss:	2.4470	Accuracy:	0.1247	F1:	0.0185
No improvement for 3 epochs. Stopping early.
best (higgest macro f1-score) val results:
                         precision    recall  f1-score   support

             ad hominem       0.00      0.00      0.00        67
               anecdote       0.00      0.00      0.00        43
         cherry picking       0.00      0.00      0.00        56
      conspiracy theory       0.00      0.00      0.00        39
           fake experts       0.00      0.00      0.00        12
           false choice       0.00      0.00      0.00        13
      false equivalence       0.00      0.00      0.00        14
impossible expectations       0.00      0.00      0.00        37
      misrepresentation       0.00  

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v2-xlarge and are newly initialized: ['classifier.weight', 'pooler.dense.weight', 'pooler.dense.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


1 / 30: Train Loss:	21.8202	Val Loss:	2.4792	Accuracy:	0.1050	F1:	0.0430 *
2 / 30: Train Loss:	21.5403	Val Loss:	2.4746	Accuracy:	0.1050	F1:	0.0468 *
3 / 30: Train Loss:	21.3193	Val Loss:	2.4705	Accuracy:	0.1116	F1:	0.0517 *
4 / 30: Train Loss:	21.2921	Val Loss:	2.4683	Accuracy:	0.1204	F1:	0.0603 *
5 / 30: Train Loss:	21.1363	Val Loss:	2.4666	Accuracy:	0.1138	F1:	0.0603
6 / 30: Train Loss:	21.1976	Val Loss:	2.4650	Accuracy:	0.1204	F1:	0.0652 *
7 / 30: Train Loss:	21.2473	Val Loss:	2.4629	Accuracy:	0.1204	F1:	0.0752 *
8 / 30: Train Loss:	20.9751	Val Loss:	2.4610	Accuracy:	0.1313	F1:	0.0893 *
9 / 30: Train Loss:	20.9785	Val Loss:	2.4598	Accuracy:	0.1313	F1:	0.0832
10 / 30: Train Loss:	21.0905	Val Loss:	2.4594	Accuracy:	0.1291	F1:	0.0843
11 / 30: Train Loss:	20.9729	Val Loss:	2.4587	Accuracy:	0.1204	F1:	0.0758
No improvement for 3 epochs. Stopping early.
best (higgest macro f1-score) val results:
                         precision    recall  f1-score   support

             ad hominem    

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v2-xlarge and are newly initialized: ['classifier.weight', 'pooler.dense.weight', 'pooler.dense.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


1 / 30: Train Loss:	21.8170	Val Loss:	2.4787	Accuracy:	0.1028	F1:	0.0414 *
2 / 30: Train Loss:	21.5306	Val Loss:	2.4738	Accuracy:	0.1050	F1:	0.0467 *
3 / 30: Train Loss:	21.3093	Val Loss:	2.4694	Accuracy:	0.1138	F1:	0.0552 *
4 / 30: Train Loss:	21.2677	Val Loss:	2.4671	Accuracy:	0.1182	F1:	0.0605 *
5 / 30: Train Loss:	21.1078	Val Loss:	2.4653	Accuracy:	0.1182	F1:	0.0633 *
6 / 30: Train Loss:	21.1736	Val Loss:	2.4634	Accuracy:	0.1247	F1:	0.0772 *
7 / 30: Train Loss:	21.2132	Val Loss:	2.4605	Accuracy:	0.1335	F1:	0.0838 *
8 / 30: Train Loss:	20.9446	Val Loss:	2.4580	Accuracy:	0.1291	F1:	0.0816
9 / 30: Train Loss:	20.9299	Val Loss:	2.4562	Accuracy:	0.1357	F1:	0.0920 *
10 / 30: Train Loss:	21.0513	Val Loss:	2.4555	Accuracy:	0.1335	F1:	0.0921 *
11 / 30: Train Loss:	20.8897	Val Loss:	2.4534	Accuracy:	0.1335	F1:	0.0928 *
12 / 30: Train Loss:	20.8101	Val Loss:	2.4517	Accuracy:	0.1225	F1:	0.0858
13 / 30: Train Loss:	21.0208	Val Loss:	2.4491	Accuracy:	0.1225	F1:	0.0888
14 / 30: Train Loss:	20.779

In [5]:
import pandas as pd

In [6]:
pd.DataFrame(results)

,test_acc,test_f1,eval_acc,eval_f1,"rank, alpha",model
0,0.484375,0.364616,0.516411,0.389022,8,bert-base-uncased
1,0.484375,0.366146,0.514223,0.393118,16,bert-base-uncased
2,0.628906,0.602685,0.689278,0.673936,8,roberta-large
3,0.667969,0.636544,0.682713,0.671765,16,roberta-large
4,0.175781,0.098885,0.183807,0.111217,8,gpt2
5,0.386719,0.301042,0.411379,0.326773,16,gpt2
6,0.457031,0.435943,0.479212,0.487516,8,bigscience/bloom-560m
7,0.468750,0.438304,0.507659,0.514580,16,bigscience/bloom-560m
8,0.140625,0.071074,0.146608,0.083974,8,facebook/opt-350m
9,0.136719,0.066688,0.129103,0.078936,16,facebook/opt-350m
